In [1]:
import numpy as np
import pandas as pd
import glob
from pathlib import Path
import matplotlib.pyplot as plt
from scipy import stats
import os
from scipy.stats import sem
from matplotlib.lines import Line2D
import behavior.boris_extraction as boris
import pickle
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import LogNorm

def hex_2_rgb(hex_color): # Orange color
    rgb_color = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (1, 3, 5))
    return rgb_color

def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):   
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl 
    Returns:
        none
    """
    with open(file_name,'wb') as file:
        pickle.dump(thing_to_pickle, file)

def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):   
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, 'rb') as file:
        return(pickle.load(file))
    
novel_collection = unpickle_this('novel_collection.pkl')
cagemate_collection = unpickle_this('cagemate_collection.pkl')

In [2]:
boris_data_path = r"C:\Users\megha\Documents\Padilla-Coreano\homecage_social_ephys_vids\pilot2\boris\habit_dishabit_aggregated\*"
file_paths = glob.glob(boris_data_path)
dictOfBoris={}
df_list =[]
for file_path in file_paths:
    if file_path.endswith('.csv'):
        phase1_df = pd.read_csv(file_path)
    elif file_path.endswith('.tsv'):
        phase1_df = pd.read_csv(file_path, sep='\t')
    df_list.append(phase1_df)
    phase1_df.name = file_path
    filename_no_ext = os.path.splitext(os.path.basename(file_path))[0]
    dictOfBoris[filename_no_ext] = phase1_df

In [ ]:
def random_event_generator(start, stop, len_event, no_events):
    total_duration = stop - start
    possible_events = np.arange(int(total_duration / len_event))
    pot_events = np.random.choice(possible_events, size = (no_events), replace = False)
    pot_events = np.sort(pot_events)
    events = []
    for i in pot_events: 
        event_start = (start + (len_event * i)) * 1000
        event_stop = (event_start + (len_event * 1000))
        events.append(np.array([event_start, event_stop]))
    return(np.array(events))
behavior = ['anogenital sniffing', 'facial sniffing']
subject = ['subject']
min_iti = 1 
min_bout = 0.5

behavior_dicts = {}
cage_dicts = {}
novel_dicts = {}
for recording, df in dictOfBoris.items():
    behavior_dict = {}
    mouse_in = np.array(df[df['Behavior'] == 'mice back in']['Start (s)'].tolist())*1000
    mouse_out = np.array(df[df['Behavior'] == 'mice taken out']['Start (s)'].tolist())*1000
    
    # Get all behavior bouts
    sniffs = boris.get_behavior_bouts(df, subject, behavior, min_iti, min_bout)
    facial_sniffs = boris.get_behavior_bouts(df, subject, behavior = ['facial sniffing'], min_iti = min_iti,min_bout =  min_bout)
    anogential_sniffs = boris.get_behavior_bouts(df, subject, behavior = ['anogenital sniffing'], min_iti = min_iti, min_bout = min_bout)
    fights = boris.get_behavior_bouts(df, subject, behavior = ['fighting'], min_iti = min_iti, min_bout = min_bout)
    chases = boris.get_behavior_bouts(df, subject, behavior = ['chasing'], min_iti = min_iti, min_bout = min_bout)
    allogrooming = boris.get_behavior_bouts(df, subject, behavior = ['allogrooming'], min_iti = min_iti, min_bout = min_bout)
    positive_social = boris.get_behavior_bouts(df, subject, behavior = ['anogenital sniffing', 'facial sniffing','allogrooming'], min_iti = min_iti, min_bout = min_bout)
    aversive_social = boris.get_behavior_bouts(df, subject = ['subject', 'social_agent'], behavior = ['fighting', 'chasing'], min_iti = min_iti, min_bout = min_bout)
    # Convert to numpy arrays, ensuring empty arrays if no events
    sniffs = np.array(sniffs) if len(sniffs) > 0 else np.array([[0,0]])
    fights = np.array(fights) if len(fights) > 0 else np.array([[0,0]])
    chases = np.array(chases) if len(chases) > 0 else np.array([[0,0]])
    allogrooming = np.array(allogrooming) if len(allogrooming) > 0 else np.array([[0,0]])
    positive_social = np.array(positive_social) if len(positive_social) > 0 else np.array([[0,0]])
    aversive_social = np.array(aversive_social) if len(aversive_social) > 0 else np.array([[0,0]])

    if (recording == '21_nov_p1_aggregated') | (recording == '31_nov_p1_aggregated'):
        if len(sniffs) > 0 and sniffs[0].size > 0:
            sniffs = np.array([[x*2 for x in sniff] for sniff in sniffs])
        if len(fights) > 0 and fights[0].size > 0:
            fights = np.array([[x*2 for x in fight] for fight in fights])
        if len(chases) > 0 and chases[0].size > 0:
            chases = np.array([[x*2 for x in chase] for chase in chases])
        if len(allogrooming) > 0 and allogrooming[0].size > 0:
            allogrooming = np.array([[x*2 for x in groom] for groom in allogrooming])
        if len(positive_social) > 0 and positive_social[0].size > 0:
            positive_social = np.array([[x*2 for x in pos] for pos in positive_social])
        if len(aversive_social) > 0 and aversive_social[0].size > 0:
            aversive_social = np.array([[x*2 for x in av] for av in aversive_social])
        mouse_in = mouse_in * 2
        mouse_out = mouse_out * 2

    # Generate baseline events
    behavior_dict['baseline'] = random_event_generator(start=0, stop=mouse_in[0]/1000, len_event=3, no_events=15)
    behavior_dict['positive'] = positive_social
    behavior_dict['aversive'] = aversive_social
    def filter_events(events, start_time, end_time=None):
        if events.size == 0 or events[0].size == 0:
            return np.array([[]])
        mask = (events[:,0] > start_time)
        if end_time is not None:
            mask = mask & (events[:,1] < end_time)
        filtered_events = events[mask]
        return filtered_events if filtered_events.size > 0 else np.array([[]])

    if recording == '22_nov_p1_aggregated':
        for exp_num in range(1, 6):
            if exp_num < 5:
                start_idx = exp_num - 1
                end_idx = exp_num - 1
                end_time = mouse_out[end_idx]
            else:
                start_idx = 4
                end_time = None
                
            behavior_dict[f'exp{exp_num} sniff'] = filter_events(sniffs, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} fight'] = filter_events(fights, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} chase'] = filter_events(chases, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} allogroom'] = filter_events(allogrooming, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} positive'] = filter_events(positive_social, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} aversive'] = filter_events(aversive_social, mouse_in[start_idx], end_time)
    else:
        for exp_num in range(1, 6):
            if exp_num < 5:
                start_idx = exp_num - 1
                end_idx = exp_num - 1
                end_time = mouse_out[end_idx]
            else:
                start_idx = 4
                end_time = None
                
            behavior_dict[f'exp{exp_num} sniff'] = filter_events(sniffs, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} fight'] = filter_events(fights, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} chase'] = filter_events(chases, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} allogroom'] = filter_events(allogrooming, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} positive'] = filter_events(positive_social, mouse_in[start_idx], end_time)
            behavior_dict[f'exp{exp_num} aversive'] = filter_events(aversive_social, mouse_in[start_idx], end_time)

    behavior_dicts[recording] = behavior_dict
    if "cage" in recording:
        cage_dicts[recording] = behavior_dict
    else:
        novel_dicts[recording] = behavior_dict

In [ ]:
color_dict = {'exp1': 'r', 'exp2': 'b', 'exp3': 'b', 'exp4': 'g',
              'exp5': 'k', 'baseline': 'y'}
recording_number = 0
plt.figure(figsize=(10, 10))
labels = []

for name, recording in behavior_dicts.items():
    labels.append(name)
    for event, snippets in recording.items():
        # Only process events containing 'positive' or 'aversive'
        if 'positive' in event or 'aversive' in event:
            #if len(snippet) 
            if event == 'positive' or event == 'aversive':
                    pass
            else:
                for i in range(len(snippets)):
                    x = snippets[i]/1000/60
                    if len(x) > 0:
            # Plot the line segments
                        if ('aversive' in event):
                            # Plot taller bars for aversive interactions
                            y = [recording_number, recording_number + 0.3]
                            plt.plot(x, y, marker='', linestyle='-', c=color_dict[event.split()[0]], linewidth=2)
                        if ('positive' in event):
                            # Plot regular lines for positive interactions
                            y = [recording_number,recording_number]
                            plt.plot(x, y, marker='o', linestyle='-', c=color_dict[event.split()[0]], markersize=1)
        
    recording_number += .5

plt.xlabel('Time (minutes)')
plt.ylabel('Recordings')
plt.yticks(ticks=np.arange(len(labels))/2, labels=labels)
plt.title('Social Interactions Timeline')

# Create legend entries for positive and aversive interactions
legend_labels = []
for label in color_dict:
    # Add line for positive interactions
    legend_labels.append(plt.Line2D([0], [0], color=color_dict[label], marker='o', 
                                  linestyle='-', markersize=1, label=f'{label} positive'))
    # Add bar for aversive interactions
    legend_labels.append(plt.Line2D([0], [0], color=color_dict[label], 
                                  linestyle='-', linewidth=2, label=f'{label} aversive'))

plt.legend(handles=legend_labels)
plt.grid(True)
plt.xlim([0, 45])
plt.show()

In [ ]:


def plot_raster_whole_recording(spike_collection):
    arrays = []
    for recording in spike_collection.collection:
        arrays.append(recording.unit_firing_rate_array)
    min_length = min(arr.shape[0] for arr in arrays)
# Trim all arrays to this length
    trimmed_arrays = []
    for unit_array in arrays:
        trimmed = unit_array[:min_length, :]
        trimmed_arrays.append(trimmed)
    firing_rate_array = np.hstack(trimmed_arrays)
    print(firing_rate_array.shape)
    #time x neurons= samples by features
    scaler = StandardScaler()
    zscore_matrix = scaler.fit_transform(firing_rate_array)
    other_way = scaler.fit_transform(firing_rate_array)
    #row_means = np.mean(firing_rate_array, axis=1)
    #sort_indices = np.argsort(row_means)[::-1]

    # Reorder the data
    #sorted_zscore = zscore_matrix[sort_indices]
    timebin_s = spike_collection.timebin / 1000 / 60 # Convert timebin from ms to seconds
    total_bins = zscore_matrix.shape[1] 
    time_axis = np.linspace(0, min_length/600, total_bins)

    # Create the plot
    fig, ax = plt.subplots(figsize=(30, 10))

    # Plot heatmap
    im = ax.imshow(other_way.T, aspect="auto", cmap="viridis", extent=[time_axis[0], time_axis[-1], zscore_matrix.shape[0], 0], vmax = 5)
    
    # Add event rectangle (from 0 to event_length in seconds)
    import matplotlib.patches as patches

 

    # Customize plot
    plt.colorbar(im, label="Z-score")
    ax.set_xlabel("Time (minutes)")
    ax.set_ylabel("Units (sorted by mean activity)")
    ax.set_title("Neural Activity Aligned to Event Onset")

  

    plt.tight_layout()
    plt.show()
plot_raster_whole_recording(cagemate_collection)
plot_raster_whole_recording(novel_collection)